# 2.Data Understanding

This section dives deep into the microfinance institution's data we received. Our goal is to gain a comprehensive understanding of the data's characteristics, including its quality, completeness, and potential issues.

* **Data Sources:** We'll identify the various sources of the data, such as loan applications, repayment records, customer demographics, and potentially external sources (if available). 
* **Data Description:** We'll provide a detailed description of each data variable, including its name, data type (numerical, categorical, etc.), and a clear definition of its meaning.
* **Data Quality Assessment:**  We'll assess the quality of the data, checking for missing values, inconsistencies, and errors. This might involve techniques like identifying outliers, checking for data type inconsistencies, and analyzing the distribution of missing values across different variables.

* **Exploratory Data Analysis (EDA):**  Here, we'll perform various statistical analyses to get a sense of the data's distribution. This could involve calculating summary statistics like mean, median, standard deviation, and frequency tables for categorical variables.  We'll also visualize the data using techniques like histograms, boxplots, and scatterplots to identify patterns, trends, and potential outliers.

We will write about *EDA* in the **Data Preparation** section since based on the workflow, We prepare the data along with exploring it.

The main table that we use to analyze and train our model, is the train dataframe which consists of 122 columns (features), each describing a criteria which has been used in evaluating the customers.

In [1]:
import os
import warnings

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
from IPython.display import clear_output

import matplotlib.pyplot as plt
import seaborn as sns

os.chdir("../")

from src import micro

%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")
clear_output()


In [2]:
train_df = micro.read_parquet_file(dir_name="original_data", file_name="application_train")
train_cols = train_df.columns
print(train_df.info(show_counts=True))
display(train_df.head(5))

# REGION_POPULATION_RELATIVE = Normalized population of region where client lives (higher number means the client lives in more pop...
# WEEKDAY_APPR_PROCESS_START = On which day of the week did the client apply for the loan
# HOUR_APPR_PROCESS_START = Approximately at what hour did the client apply for the loan

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB
None


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


At first we need to make sure that the columns which are present in the training dataset are also present in the test data set too. We leave the "TARGET" column to be, and we will keep the common columns between train and test dataset.

In [5]:
test_df = micro.read_parquet_file(dir_name="original_data", file_name="application_test")

# Since we have a big DataFrame we need to make sure that we work on the columns that are present in the test DataFrame too.
# Find the shared columns
shared_data_cols = [col for col in train_cols if col in test_df.columns]

# Add the TARGET Column which isn't available in the test DataFrame
shared_data_cols.append("TARGET")

# Select the shared columns in train dataframe
train_df = train_df.loc[:, shared_data_cols]

display(train_df.info(show_counts=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float64(65), int64(41), object(16)
memory usage: 286.2+ MB


None

Based on the code below we see that we have about 67 Columns with missing values which needs to be taken care of later. The values of number of missing values are show as percentage, we can see that some of them even have NA values up to 70% of the rows.

We have to be delicate into handling them.

In [23]:
micro.find_na_cols(train_df, na_thresh=100)

COMMONAREA_MEDI             69.872297
COMMONAREA_AVG              69.872297
COMMONAREA_MODE             69.872297
NONLIVINGAPARTMENTS_MEDI    69.432963
NONLIVINGAPARTMENTS_MODE    69.432963
                              ...    
EXT_SOURCE_2                 0.214626
AMT_GOODS_PRICE              0.090403
AMT_ANNUITY                  0.003902
CNT_FAM_MEMBERS              0.000650
DAYS_LAST_PHONE_CHANGE       0.000325
Length: 67, dtype: float64 

The information about the NA columns: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 67 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   AMT_ANNUITY                   307499 non-null  float64
 1   AMT_GOODS_PRICE               307233 non-null  float64
 2   NAME_TYPE_SUITE               306219 non-null  object 
 3   OWN_CAR_AGE                   104582 non-null  float64
 4   OCCUPATION_TYPE               211120 

We can see that there 61 columns which ha the type of *float* and 6 columns has the *object* type which could be strings.

We have to use different methods for each type of columns even with different number of missing values in order to handle the NA values. We will take care these NA values in the **Data Preparation** section.